# Missing Values

In this tutorial, you will learn three approaches to **dealing with missing values**. Then you'll compare the effectiveness of these approaches on a real-world dataset.

## Introduction

There are many ways data can end up with missing values. For example,
* A 2 bedroom house won't include a value for the size of a third bedroom.
* A survey respondent may choose not to share his income.

Most machine learning libraries (including `scikit-learn`) give an error if you try to build a model using missing values. So you'll need to choose one of the strategies below.

## Approaches to missing values

### 1. A simple option: drop columns with missing values.

The simplest option is to drop columns with missing values.

Unless most values in the dropped column are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

### 2. A better option: imputation

**Imputation** fills in the missing values with some number. For instance, we can fill in the mean value along each column.

The imputed values won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

### 3. An extension to imputation

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

## Example

In this example, we will work with the [Melbourne Housing dataset](https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot). Our model will use information such as the number of rooms and land size to predict home price.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv("./input/melb_data.csv")

# Select target
y = data.Price

# Only use numerical predictors for simplicity
melb_predictors = data.drop(["Price"], axis=1)
X = melb_predictors.select_dtypes(exclude=["object"])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

### Define function to measure quality of each approach

We define a function `score_dataset()` to compare different approaches to dealing with missing values. This function reports the mean absolute error (MAE) from a random forest model.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Score from approach 1 (drop columns with missing values)

Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.

In [3]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from approach 1 (drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from approach 1 (drop columns with missing values):
183550.22137772635


### Score from approach 2 (imputation)

Next, we use `SimpleImputer` to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as **regression imputation**, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [4]:
from sklearn.impute import SimpleImputer

# Make an instance of the imputer
my_imputer = SimpleImputer()
# fit_transform() does two things:
    # 1. fit: calculate the imputative values (e.g. mean of each column in the X_train)
    # 2. transform: replaced the missing values in X_train using those values
# The result is returned as a NumPy array so pd.DataFrame() converts it back to a DataFram
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# transform() applies the imputation using the values learned from X_train
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removes column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from approach 2 (imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from approach 2 (imputation):
178166.46269899711


We see that approach 2 has a lower MAE than approach 1, so approach 2 performed better on this dataset.

### Score from approach 3 (an extension to imputation)

Next, we impute the missing values, while also keeping track of which values were imputed.

In [5]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing: # For each column with missing data:
    # Create a new column called "col-name_was_missing"
    # .isnull() checks each value and returns True if it was missing, False if not
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from approach 3 (an extension to imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from approach 3 (an extension to imputation):
178927.503183954


As we can see, approach 3 preformed slightly worse than approach 2.

So, why did imputation perform better than dropping the columns?

The training data has 10,864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [6]:
# Shape of training data
print(X_train.shape)

# Number of missing values in each column of the training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


## Conclusion

As is common, imputing missing values (in approach 2 and approach 3) yielded better results, relative to when we simply dropped columns with missing values (approach 1).